In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.io as sio
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define PINN with classifier
class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_classes):
        super(PINN, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes),
            nn.Softmax(dim=1)
        )
        self.regressor = nn.Sequential(
            nn.Linear(input_size + num_classes, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        class_probs = self.classifier(x)  # Ensure classifier gets the right input
        x_combined = torch.cat((x, class_probs), dim=1)  # Ensure correct concatenation
        return self.regressor(x_combined), class_probs

# Load and preprocess data
def load_data(directory):
    X, y, labels = [], [], []
    for filename in os.listdir(directory):
        if filename.endswith(".mat"):
            data = sio.loadmat(os.path.join(directory, filename))
            wind_angle = data['Wind_azimuth'][0][0]  # Extract from file
            height_ratio = data['Building_height'][0][0] / data['Building_depth'][0][0]  # Compute height-to-depth ratio
            Cp_mean = np.mean(data['Wind_pressure_coefficients'], axis=1)  # Extract mean wind pressure coefficient
            X.append([wind_angle, height_ratio])
            y.append(Cp_mean)
            labels.append(data['Roof_type'][0])  # Extract roof type
    return np.array(X), np.array(y), np.array(labels)

# Prepare dataset
directory = "../data/DATAAIO"
X, y, labels = load_data(directory)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
labels_encoded = encoder.fit_transform(labels.reshape(-1, 1))

# Combine X with labels
X_combined = np.hstack((X, labels_encoded))

# Standardize data
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_combined = scaler_X.fit_transform(X_combined)
y = scaler_y.fit_transform(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Convert to tensors and move to device
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

# Check input dimensions
print("X_train shape:", X_train.shape)  # Should be (num_samples, num_features)
print("y_train shape:", y_train.shape)
print("Labels encoded shape:", labels_encoded.shape)

# Model setup
input_size = X.shape[1]  # The number of features *before* one-hot encoding
hidden_size = 64
output_size = y_train.shape[1]  # Number of Cp values per sample
num_classes = labels_encoded.shape[1]  # Number of roof types

model = PINN(input_size + num_classes, hidden_size, output_size, num_classes).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
epochs = 10000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs, _ = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}')

# Save model
torch.save(model.state_dict(), "pinn_model.pth")
print("Model training complete and saved as pinn_model.pth")

# Validation
model.eval()
with torch.no_grad():
    y_pred, _ = model(X_test)
    y_pred = y_pred.cpu().numpy()
    y_test = y_test.cpu().numpy()

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print results
print(f'🔎 Validation Results:')
print(f'✅ MAE: {mae:.4f}')
print(f'✅ MSE: {mse:.4f}')
print(f'✅ RMSE: {rmse:.4f}')
print(f'✅ R² Score: {r2:.4f}')



X_train shape: torch.Size([48, 3])
y_train shape: torch.Size([48, 14063])
Labels encoded shape: (60, 1)
Epoch [0/10000], Loss: 1.0027
Epoch [100/10000], Loss: 0.9831
Epoch [200/10000], Loss: 0.9558
Epoch [300/10000], Loss: 0.9238
Epoch [400/10000], Loss: 0.8980
Epoch [500/10000], Loss: 0.8756
Epoch [600/10000], Loss: 0.8545
Epoch [700/10000], Loss: 0.8354
Epoch [800/10000], Loss: 0.8191
Epoch [900/10000], Loss: 0.8048
Epoch [1000/10000], Loss: 0.7924
Epoch [1100/10000], Loss: 0.7811
Epoch [1200/10000], Loss: 0.7708
Epoch [1300/10000], Loss: 0.7611
Epoch [1400/10000], Loss: 0.7517
Epoch [1500/10000], Loss: 0.7424
Epoch [1600/10000], Loss: 0.7333
Epoch [1700/10000], Loss: 0.7245
Epoch [1800/10000], Loss: 0.7160
Epoch [1900/10000], Loss: 0.7079
Epoch [2000/10000], Loss: 0.7003
Epoch [2100/10000], Loss: 0.6933
Epoch [2200/10000], Loss: 0.6869
Epoch [2300/10000], Loss: 0.6813
Epoch [2400/10000], Loss: 0.6764
Epoch [2500/10000], Loss: 0.6723
Epoch [2600/10000], Loss: 0.6690
Epoch [2700/10000